In [1]:
from trajectree.fock_optics.utils import create_vacuum_state
from trajectree.fock_optics.devices import global_phase
from trajectree.quant_info.noise_models import amplitude_damping
from trajectree.quant_info.circuit import Circuit
from trajectree.trajectory import quantum_channel, trajectory_evaluator
import numpy as np
from scipy import sparse as sp
from matplotlib import pyplot as plt
import cirq
import qsimcirq
import time

from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import EstimatorV2 as Estimator
from qiskit import transpile, QuantumCircuit
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit.primitives import SamplerResult
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    amplitude_damping_error,
    thermal_relaxation_error,
)

from mqt.bench import BenchmarkLevel, get_benchmark

In [2]:
num_qubits = 1
num_trajectories = 100
noise_probability = np.sqrt(0.5)

qc = QuantumCircuit(num_qubits)
qc.h(0)
# qc.cx(0,1)
# qc.x(0)

In [3]:
# print(dumps(qc))
target_basis = ['h', 's', 'x', 't', 'ry', 'rz', 'cx', 'cz']
transpiled_qc_custom = transpile(qc, basis_gates=target_basis, optimization_level=1)

noise_channel = amplitude_damping_error(noise_probability)

two_qubit_noise_channel = noise_channel.tensor(noise_channel)

noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(noise_channel, target_basis[:-2])
noise_model.add_all_qubit_quantum_error(two_qubit_noise_channel, ["cx", 'cz'])

print(noise_model)
transpiled_qc_custom.draw(output="text")

NoiseModel:
  Basis gates: ['cx', 'cz', 'h', 'id', 'ry', 'rz', 's', 'sx', 't', 'x']
  Instructions with noise: ['h', 'cx', 'x', 'cz', 's', 'rz', 't', 'ry']
  All-qubits errors: ['h', 's', 'x', 't', 'ry', 'rz', 'cx', 'cz']


┌───┐
q: ┤ H ├
   └───┘

In [4]:

observable = SparsePauliOp("Z" * (num_qubits))

# Use the Sampler primitive for latest API
backend = AerSampler()

noisy_estimator = Estimator(options=dict(backend_options=dict(noise_model=noise_model), run_options=dict(shots=num_trajectories)))

pub = (transpiled_qc_custom, observable)

job = noisy_estimator.run([pub])
result = job.result()
pub_result = result[0]
# print((len(result)))
print(float(pub_result.data.evs))
print(float(pub_result.data.stds))

0.7071067811865479
0.0


In [5]:
qc = Circuit(num_qubits, backend = 'tensor')
qc.qiskit_to_trajectree(transpiled_qc_custom, noise_parameter=noise_probability)
qc.expectation('z'*(num_qubits))
evs_list = []
for i in range(1):
    evs, times = qc.perform_trajectree_simulation(num_trajectories, error_tolerance = 1e-16)
    evs_list.append(np.mean(evs))
print("outside of simulation")
# evs
np.mean(evs_list)
# evs

time taken: 0.009407758712768555
time taken: 0.0021266937255859375
time taken: 0.0009002685546875
time taken: 0.001016378402709961
time taken: 0.000995635986328125
time taken: 0.000972747802734375
time taken: 0.0010991096496582031
time taken: 0.00028324127197265625
time taken: 0.0005779266357421875
time taken: 0.00021910667419433594
time taken: 0.0005717277526855469
time taken: 0.0007047653198242188
time taken: 0.00021696090698242188
time taken: 0.0006115436553955078
time taken: 0.0002219676971435547
time taken: 0.0006768703460693359
time taken: 0.0002193450927734375
time taken: 0.0006365776062011719
time taken: 0.0006880760192871094
time taken: 0.00021600723266601562
time taken: 0.000659942626953125
time taken: 0.00021195411682128906
time taken: 0.00020575523376464844
time taken: 0.0009291172027587891
time taken: 0.0021719932556152344
time taken: 0.0007357597351074219
time taken: 0.000701904296875
time taken: 0.00021696090698242188
time taken: 0.0006339550018310547
time taken: 0.00022

time taken: 0.0013849735260009766
time taken: 0.0007052421569824219
time taken: 0.00022864341735839844
time taken: 0.0006086826324462891
time taken: 0.00021505355834960938
time taken: 0.0006680488586425781
time taken: 0.0007021427154541016
time taken: 0.00021314620971679688
time taken: 0.0007035732269287109
time taken: 0.00020956993103027344
time taken: 0.0006864070892333984
time taken: 0.0006654262542724609
time taken: 0.00021219253540039062
time taken: 0.0006680488586425781
time taken: 0.00021457672119140625
time taken: 0.0006906986236572266
time taken: 0.00021767616271972656
time taken: 0.0012025833129882812
time taken: 0.00028395652770996094
time taken: 0.0006041526794433594
time taken: 0.0007634162902832031
time taken: 0.0008862018585205078
time taken: 0.0006773471832275391
time taken: 0.0006537437438964844
time taken: 0.0006971359252929688
time taken: 0.0002384185791015625
time taken: 0.0006964206695556641
time taken: 0.0006902217864990234
time taken: 0.00021576881408691406
time 

np.float64(0.682842712474619)

In [65]:
import numpy as np
l = [1, 0.54691]
result = 0
num_samples = 1000000
for i in range(num_samples):
    sample = np.random.choice(a=l, p=[0.35355, 1-0.35355])
    result += sample
print("estimatee:", result/num_samples)
print("exact:", 0.35355 * l[0] + (1-0.35355)*l[1])


estimatee: 0.7070637223005085
exact: 0.7070999695


In [ ]:
0.35355 + 0.64644

0.9999900000999999

In [6]:
print(qc.t_eval.trajectree[1][(0,)].trajectories)
print(qc.t_eval.trajectree[1][(0,)].weights)
print(qc.t_eval.trajectree[1][(0,)].trajectory_indices)

[1.0 0.5469181606780272]
[0.64644661 0.35355339]
[1 0]
